In [2]:
import sys
sys.path.append('/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage')

from main import parser_arguments, regional_extremes_method, local_extremes_method, regional_extremes_minicube

In [3]:
import numpy as np
from dask_jobqueue import SLURMCluster
from dask.distributed import Client as daskClient

In [4]:
# Set distributed scheduler as default
import dask
dask.config.set(scheduler='distributed')

cluster = SLURMCluster(
    queue='work',                  # Specify the SLURM queue
    processes=1,                     # Number of processes per job
    cores=1,                          # Number of cores per job
    memory='50GB',                    # Memory per job
    walltime='15:00:00',              # Job duration (hh:mm:ss)
    job_script_prologue=[
        'module load BGC-easybuilded',
        'module load  GCC',
    ]
)

# Scale up the number of workers
#cluster.scale(jobs=20)  # Adjust the number of jobs/workers
cluster.adapt(minimum=0, maximum=30)

# Create a Dask client that connects to the cluster
client = daskClient(cluster)

# Check cluster status
cluster

/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42325 instead
  warnings.warn(


Dashboard: http://10.0.30.56:42325/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.30.56:36533,Workers: 0
Dashboard: http://10.0.30.56:42325/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
import sys
import argparse

# Remove Jupyter's arguments before parsing your own
sys.argv = sys.argv[:1]


args = parser_arguments().parse_args()
args

Namespace(id=None, name=None, index='pei_180', compute_variance=False, region='globe', time_resolution=5, n_components=3, n_samples=100, n_eco_clusters=25, kernel_pca=False, saving_path=None, path_load_experiment=None, method='regional')

In [6]:
import numpy as np 

# Remove Jupyter's arguments before parsing your own
sys.argv = sys.argv[:1]
args = parser_arguments().parse_args()
args.name = "deep_extreme_HR"
args.index = "EVI_EN"
args.k_pca = False
args.n_samples = 20000
args.n_components = 3
args.n_eco_clusters = 50
args.compute_variance = False
args.method = "regional"
args.start_year = 2000
args.is_generic_xarray_dataset = False

# args.path_load_experiment = "/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-12_11:52:00_deep_extreme_HR"

LOWER_QUANTILES_LEVEL = np.array([0.01, 0.025, 0.05])
UPPER_QUANTILES_LEVEL = np.array([0.95, 0.975, 0.99])
if args.method == "regional":
    # Apply the regional extremes method
    # regional_extremes_method(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
    regional_extremes_minicube(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
elif args.method == "local":
    # Apply the uniform threshold method
    local_extremes_method(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
elif args.method == "global":
    raise NotImplementedError("the global method is not yet implemented.")



[2024-12-19 13:52:48] Initialisation of a new model, no path provided for an existing model.
[2024-12-19 13:52:48] The saving path is: /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-19_13:52:48_deep_extreme_HR/EVI_EN
[2024-12-19 13:52:48] args saved, path: /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-19_13:52:48_deep_extreme_HR/EVI_EN/args.json
[2024-12-19 13:52:48] start of the preprocess
/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-19_13:52:48_deep_extreme_HR/EVI_EN/temp_file.zarr
[2024-12-19 13:52:48] Data not found at /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-19_13:52:48_deep_extreme_HR/EVI_EN/temp_file.zarr
count: 20000
[2024-12-19 13:52:49] Loading dataset...


/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/dask/base.py:1103: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


[########################################] | 100% Completed | 520.39 s


/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 843.92 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


[2024-12-19 14:25:53] temp_file computed and saved.
/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-19_13:52:48_deep_extreme_HR/EVI_EN/temp_file.zarr
temp_file.zarr loaded.
[2024-12-19 14:25:59] Dataset loaded.
[2024-12-19 14:26:02] Computation on the entire dataset. 10721 samples
[2024-12-19 14:27:01] msc computed and saved.
[2024-12-19 14:27:01] PCA performed. Sum of explained variance: 0.9258382655745034.Explained variance ratio: [0.64413595 0.24220317 0.03949914].
[2024-12-19 14:27:01] PCA saved: /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-19_13:52:48_deep_extreme_HR/EVI_EN/pca_matrix.pkl
[2024-12-19 14:27:01] start of the preprocess
/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-19_13:52:48_deep_extreme_HR/EVI_EN/temp_file.zarr
temp_file.zarr loaded.
[2024-12-19 14:27:01] Computation on the entire dataset

...

/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 61.85 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


[2024-12-19 14:35:46] thresholds_locations computed and saved.


/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 76.35 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 40.49 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


[2024-12-19 14:37:32] extremes computed and saved.


: 

In [7]:
client.close()
cluster.close()